In [2]:
import os
import cv2
import numpy as np
import glob

# Import of Metavision Machine Learning binding
import metavision_sdk_ml
import metavision_sdk_cv
from metavision_sdk_core import EventBbox
from metavision_sdk_core import BaseFrameGenerationAlgorithm
from metavision_core.event_io.py_reader import EventDatReader
from metavision_core.event_io.events_iterator import EventsIterator
from metavision_core.event_io import EventNpyReader

%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [11, 7]

In [2]:
event_dat_dir = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/detection_dataset_duration_60s_ratio_1.0/test"
gt_label_dir = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl"
dt_label_dir = "/home/tkyen/opencv_practice/HMNet/experiments/detection/workspace/hmnet_B3_yolox/result/pred_test_10"
video_dir = dt_label_dir + "_video"
os.makedirs(video_dir, exist_ok=True)

event_dat_list = sorted(glob.glob(os.path.join(event_dat_dir, "*.dat")))
gt_label_list = sorted(glob.glob(os.path.join(gt_label_dir, "*.npy")))
dt_label_list = sorted(glob.glob(os.path.join(dt_label_dir, "*.npy")))

In [3]:
event_dat_dir = "/home/tkyen/opencv_practice/data_3/Gen1_Automotive/detection_dataset_duration_60s_ratio_1.0/test_aug"
gt_label_dir = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet_aug/test_lbl"
dt_label_dir = "/home/tkyen/opencv_practice/HMNet/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test_hmnet_aug"
video_dir = dt_label_dir + "_video"
os.makedirs(video_dir, exist_ok=True)

event_dat_list = sorted(glob.glob(os.path.join(event_dat_dir, "*.dat")))
gt_label_list = sorted(glob.glob(os.path.join(gt_label_dir, "*.npy")))
dt_label_list = sorted(glob.glob(os.path.join(dt_label_dir, "*.npy")))

In [4]:
for i, (event_dat_path, gt_label_path, dt_label_path) in enumerate(zip(event_dat_list, gt_label_list, dt_label_list)):
    event_file = os.path.basename(event_dat_path).replace('_td.dat', '')
    gt_file = os.path.basename(gt_label_path).replace('_bbox.npy', '')
    dt_file = os.path.basename(dt_label_path).replace('_bbox.npy', '')
    assert event_file==gt_file==dt_file

    event_dat = EventDatReader(event_dat_path)
    event_dat.seek_time(0)
    ev_height, ev_width = event_dat.get_size()

    gt_label = EventNpyReader(gt_label_path)
    gt_label.seek_time(0)

    dt_label = EventNpyReader(dt_label_path)
    dt_label.seek_time(0)

    delta_t = 50000

    # VideoWriter
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    frame_rate = 10
    video_writer = cv2.VideoWriter(os.path.join(video_dir, '{}.mp4'.format(event_file)), fourcc, frame_rate, (ev_width*2, ev_height))
    colors = [[0,0,255], [0,255,0]]

    for i in range(int(60e6//delta_t)):
        events = event_dat.load_delta_t(delta_t=delta_t)
        gt_boxes = gt_label.load_delta_t(delta_t=delta_t)
        dt_boxes = dt_label.load_delta_t(delta_t=delta_t)

        image_all = np.zeros((ev_height, ev_width*2, 3), dtype=np.uint8)
        BaseFrameGenerationAlgorithm.generate_frame(events, image_all[:,:ev_width, :])
        image_all[:,ev_width:, :] = image_all[:,:ev_width, :].copy()
        cv2.putText(image_all, '{} ms / {} ms'.format(i*delta_t//1000, (i+1)*delta_t//1000), 
                    (ev_width-200, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image_all, 'GT',
                    (10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image_all, 'DT',
                    (ev_width+10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        image_all = cv2.line(image_all, (ev_width,0), (ev_width,ev_height), (0,255,255), 1)

        for gt_box in gt_boxes:
            t, x, y, w, h, class_id, confidence, track_id, invalid = gt_box
            x, y, w, h = int(x), int(y), int(w), int(h)
            if invalid == False:
                cv2.rectangle(image_all, (x, y), (x+w, y+h), colors[class_id], 1)

        for dt_box in dt_boxes:
            t, x, y, w, h, class_id, track_id, confidence = dt_box
            if confidence > 0.4:
                x, y, w, h = int(x), int(y), int(w), int(h)
                cv2.rectangle(image_all, (ev_width+x, y), (ev_width+x+w, y+h), colors[class_id], 1)

        video_writer.write(image_all)
    video_writer.release()
    # break

In [29]:
dt_label = EventNpyReader(dt_label_path)
dt_label.seek_time(0)

In [32]:
while 1:
    dt_boxes = dt_label.load_delta_t(5000)
    if len(dt_boxes) > 0:
        print(dt_boxes)
        break

[(60000, 103.47125, 139.07794, 57.60663 , 33.27533 , 0, 0, 0.9034182 )
 (60000, 264.98108, 122.42304, 39.01892 , 65.67785 , 0, 0, 0.02851164)
 (60000, 262.36133, 130.877  , 40.019714, 48.435944, 0, 0, 0.01577923)]


In [33]:
gt_label = EventNpyReader(gt_label_path)
gt_label.seek_time(0)

In [35]:
while 1:
    gt_boxes = gt_label.load_delta_t(5000)
    if len(gt_boxes) > 0:
        print(gt_boxes)
        break

[(599999, 0., 126., 110., 81., 0, 1., 272, False)]


In [21]:
ev_height, ev_width = event_dat.get_size()
event_dat.seek_time(0)
events = event_dat.load_delta_t(delta_t=delta_t)

In [36]:
ev_height
ev_width

304

In [22]:
record_dat = EventDatReader("/home/tkyen/opencv_practice/data_1/Gen4_Automotive/test_dat/moorea_2019-06-17_test_01_000_1525500000_1585500000_td.dat")
rc_height, rc_width = record_dat.get_size()

record_dat.seek_time(0)
record_events = record_dat.load_delta_t(delta_t=delta_t)


In [23]:
rc_height

720

In [43]:
gt_label = np.load("/home/tkyen/opencv_practice/data_1/Gen4_Automotive/test_dat/moorea_2019-04-18_test_03_000_671500000_731500000_bbox.npy")
gt_ts = np.unique(gt_label['t'])

for i in range(len(gt_ts)//8):
    print(gt_ts[i*8:(i+1)*8])
print(gt_ts[(i+1)*8:])

[  4166  37533  70900 104266 137633 171000 204366 237733]
[271100 304466 337833 371200 404566 437933 471300 504666]
[538033 571400 604766 638133 671500 704866 738233 771600]
[ 804966  838333  871700  905066  938433  971800 1005166 1038533]
[1071900 1105266 1138633 1172000 1205366 1238733 1272100 1305466]
[1338833 1372200 1405566 1438933 1472300 1505666 1539033 1572400]
[1605766 1639133 1672500 1705866 1739233 1772600 1805966 1839333]
[1872700 1906066 1939433 1972800 2006166 2039533 2072900 2106266]
[2139633 2173000 2206366 2239733 2273100 2306466 2339833 2373200]
[2406566 2439933 2473300 2506666 2540033 2573400 2606766 2640133]
[2673500 2706866 2740233 2773600 2806966 2840333 2873700 2907066]
[2940433 2973800 3007166 3040533 3073900 3107266 3140633 3174000]
[3207366 3240733 3274100 3307466 3340833 3374200 3407566 3440933]
[3474300 3507666 3541033 3574400 3607766 3641133 3674500 3707866]
[3741233 3774600 3807966 3841333 3874700 3908066 3941433 3974800]
[4008166 4041533 4074900 4108266 4